In [1]:
!pip install telethon sentence-transformers faiss-cpu langchain sqlite3 numpy

ERROR: Could not find a version that satisfies the requirement sqlite3 (from versions: none)
ERROR: No matching distribution found for sqlite3


In [2]:
!pip install -U langchain-text-splitters

In [3]:
import sqlite3
print(sqlite3.sqlite_version)


3.37.2


In [4]:
!pip install faiss-cpu


  Using cached faiss_cpu-1.13.0-cp39-abi3-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (7.7 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 88.7 MB/s eta 0:00:00


In [5]:
!pip install -qU --no-deps langchain langchain-text-splitters langchain-community


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 94.9 MB/s eta 0:00:00


In [7]:
import sqlite3
import json
import numpy as np
from datetime import datetime
from sentence_transformers import SentenceTransformer
import faiss
import pickle
from langchain_text_splitters import RecursiveCharacterTextSplitter

# === Инициализация multilingual модели эмбеддингов ===
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')


class RAGSystem:
    def __init__(self, db_path='rag_khanty.db'):
        """Инициализация подключения к БД и переменных"""
        self.db_path = db_path
        self.conn = sqlite3.connect(db_path)
        self.cursor = self.conn.cursor()
        self.index = None
        self.chunk_ids = []

    def create_tables(self):
        """Создание всех необходимых таблиц"""
        # Таблица документов
        self.cursor.execute('''
            CREATE TABLE IF NOT EXISTS documents (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                title TEXT NOT NULL,
                content TEXT NOT NULL,
                doc_type TEXT,
                created_at TEXT
            )
        ''')

        # Таблица чанков
        self.cursor.execute('''
            CREATE TABLE IF NOT EXISTS document_chunks (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                document_id INTEGER NOT NULL,
                chunk_text TEXT NOT NULL,
                chunk_index INTEGER NOT NULL,
                FOREIGN KEY (document_id) REFERENCES documents(id)
            )
        ''')

        # Таблица эмбеддингов (хранение в pickle-байтах)
        self.cursor.execute('''
            CREATE TABLE IF NOT EXISTS chunk_embeddings (
                chunk_id INTEGER PRIMARY KEY,
                embedding BLOB,
                FOREIGN KEY (chunk_id) REFERENCES document_chunks(id)
            )
        ''')

        # Таблица метаданных
        self.cursor.execute('''
            CREATE TABLE IF NOT EXISTS metadata (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                chunk_id INTEGER NOT NULL,
                key TEXT NOT NULL,
                value TEXT NOT NULL,
                FOREIGN KEY (chunk_id) REFERENCES document_chunks(id)
            )
        ''')

        self.conn.commit()
        print("✅ Созданы таблицы: documents, document_chunks, chunk_embeddings, metadata")

    def add_documents(self, documents_data):
        """Добавление документов: [(title, content, doc_type), ...]"""
        for title, content, doc_type in documents_data:
            self.cursor.execute(
                "INSERT INTO documents (title, content, doc_type, created_at) VALUES (?, ?, ?, ?)",
                (title, content, doc_type, datetime.now().isoformat())
            )
        self.conn.commit()
        print(f"✅ Добавлено {len(documents_data)} документов")

    def chunk_documents(self, chunk_size=200, chunk_overlap=50):
        """Разбиение документов на чанки"""
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size,
            chunk_overlap=chunk_overlap,
            length_function=len,
            separators=["\n\n", "\n", ". ", " ", ""]
        )

        self.cursor.execute("SELECT id, content FROM documents")
        documents = self.cursor.fetchall()

        total_chunks = 0
        for doc_id, content in documents:
            chunks = text_splitter.split_text(content)
            for i, chunk in enumerate(chunks):
                self.cursor.execute(
                    "INSERT INTO document_chunks (document_id, chunk_text, chunk_index) VALUES (?, ?, ?)",
                    (doc_id, chunk, i)
                )
                total_chunks += 1

        self.conn.commit()
        print(f"✅ Создано {total_chunks} чанков")

    def generate_embeddings(self):
        """Генерация эмбеддингов и создание FAISS-индекса"""
        self.cursor.execute("SELECT id, chunk_text FROM document_chunks")
        chunks = self.cursor.fetchall()

        if not chunks:
            print("⚠️ Нет чанков для векторизации")
            return

        chunk_texts = [chunk[1] for chunk in chunks]
        self.chunk_ids = [chunk[0] for chunk in chunks]

        print("🧠 Генерация эмбеддингов...")
        embeddings = model.encode(chunk_texts, convert_to_numpy=True)

        # Сохранение в БД
        for chunk_id, embedding in zip(self.chunk_ids, embeddings):
            embedding_bytes = pickle.dumps(embedding)
            self.cursor.execute(
                "INSERT OR REPLACE INTO chunk_embeddings (chunk_id, embedding) VALUES (?, ?)",
                (chunk_id, embedding_bytes)
            )

        self.conn.commit()

        # Создание FAISS индекса (косинусное сходство → IndexFlatIP + нормализация)
        dimension = embeddings.shape[1]
        self.index = faiss.IndexFlatIP(dimension)
        faiss.normalize_L2(embeddings)
        self.index.add(embeddings.astype('float32'))

        print(f"✅ FAISS индекс создан: {self.index.ntotal} векторов")

    def search(self, query, k=3):
        """Семантический поиск по запросу"""
        if self.index is None:
            print("❌ Индекс не создан. Выполните generate_embeddings().")
            return []

        query_vec = model.encode([query], convert_to_numpy=True)
        faiss.normalize_L2(query_vec)
        distances, indices = self.index.search(query_vec.astype('float32'), k)

        results = []
        for distance, idx in zip(distances[0], indices[0]):
            if idx >= len(self.chunk_ids):
                continue
            chunk_id = self.chunk_ids[idx]

            self.cursor.execute('''
                SELECT dc.chunk_text, d.title, d.doc_type
                FROM document_chunks dc
                JOIN documents d ON dc.document_id = d.id
                WHERE dc.id = ?
            ''', (chunk_id,))

            row = self.cursor.fetchone()
            if row:
                chunk_text, title, doc_type = row
                results.append({
                    'chunk_id': chunk_id,
                    'score': float(distance),
                    'text': chunk_text,
                    'document_title': title,
                    'document_type': doc_type
                })
        return results

    def close(self):
        self.conn.close()


def main():
    # === Загрузка данных из датасета ===
    documents_data = []
    try:
        with open("khantystudies_sample.jsonl", "r", encoding="utf-8") as f:
            for line in f:
                post = json.loads(line)
                title = f"Пост {post['id']}"
                content = post["text"]
                doc_type = "telegram_post"
                documents_data.append((title, content, doc_type))
    except FileNotFoundError:
        print("Файл не найден :(")
        return

    # === Инициализация RAG-системы ===
    rag_system = RAGSystem("khanty_rag.db")
    rag_system.create_tables()
    rag_system.add_documents(documents_data)
    rag_system.chunk_documents(chunk_size=200, chunk_overlap=50)
    rag_system.generate_embeddings()

    # === Тестовые запросы (на русском, по теме хантыйского языка) ===
    test_queries = [
        "хантыйский фольклор",
        "хантыйский язык",
        "традиции ханты",
        "язык ханты примеры",
        "исчезающие языки Севера"
    ]

    print("\n🔍 Тестовые запросы:\n")
    for query in test_queries:
        print(f"Запрос: '{query}'")
        results = rag_system.search(query, k=2)
        if results:
            for i, res in enumerate(results, 1):
                print(f"  {i}. [Сходство: {res['score']:.4f}] {res['text'][:120]}...")
        else:
            print("  Нет результатов")
        print()

    rag_system.close()
    print("✅ Работа завершена. База: khanty_rag.db")


if __name__ == "__main__":
    main()

✅ Созданы таблицы: documents, document_chunks, chunk_embeddings, metadata
✅ Добавлено 101 документов
✅ Создано 1103 чанков
🧠 Генерация эмбеддингов...
✅ FAISS индекс создан: 1103 векторов

🔍 Тестовые запросы:

Запрос: 'хантыйский фольклор'
  1. [Сходство: 0.7315] **Речь идет о хантах из Сургутского района...
  2. [Сходство: 0.6927] 🤩 Во-первых, не всем знакомый хантыйский контекст: ритуалы и медвежьи пляски, практика избегания с закрытием лица платко...

Запрос: 'хантыйский язык'
  1. [Сходство: 0.8138] **Речь идет о хантах из Сургутского района...
  2. [Сходство: 0.8113] **__Из книги Маины Афанасьевны Лапиной, исследовательницы и представительницы северной группы хантов. Эти названия из ка...

Запрос: 'традиции ханты'
  1. [Сходство: 0.7371] **Речь идет о хантах из Сургутского района...
  2. [Сходство: 0.7009] **Хантыйская шуба (зима близко)**

Выше я сделала репост с рассказом о традиционном саке – это верхняя женская меховая о...

Запрос: 'язык ханты примеры'
  1. [Сходство: 0.7725] 